In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
input_folder = '/content/drive/MyDrive/datasets/dataset/'


In [ ]:
splitfolders.ratio(input_folder, output='/content/drive/MyDrive/datasets/covid_dataset',
                   seed= 42, ratio=(0.8,0.1,0.1),group_prefix=None)


Copying files: 392 files [02:04,  3.15 files/s]


Start project
  

In [ ]:
TRAIN_PATH = '/content/drive/MyDrive/datasets/covid_dataset/train'
VAL_PATH = '/content/drive/MyDrive/datasets/covid_dataset/val'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image


CNN model


In [ ]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25)) 

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25)) 

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5)) 
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()  

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 64)        0

In [ ]:
##data augmntation
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range =0.2,
    zoom_range=0.2,
    horizontal_flip = True, 
)
test_datagen = image.ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/datasets/covid_dataset/train',
                                                    target_size = (224,224),
                                                    batch_size=32,
                                                    class_mode='binary')
val_generator = test_datagen.flow_from_directory('/content/drive/MyDrive/datasets/covid_dataset/val',
                                                  target_size = (224,224),
                                                  batch_size=32,
                                                  class_mode='binary')

Found 312 images belonging to 2 classes.
Found 38 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'covid': 0, 'normal': 1}

In [ ]:
 hist = model.fit_generator(
     train_geneartor,
     steps_per_epoch=8,
     epochs=30,
     validation_data= val_generator,
     validation_steps=2,   
 )

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
8/8 [==============================] - 58s 1s/step - loss: 2.3099 - accuracy: 0.5493 - val_loss: 0.6757 - val_accuracy: 0.5000
Epoch 2/10
8/8 [==============================] - 9s 1s/step - loss: 0.6773 - accuracy: 0.5956 - val_loss: 0.6305 - val_accuracy: 0.8421
Epoch 3/10
8/8 [==============================] - 10s 1s/step - loss: 0.4915 - accuracy: 0.8139 - val_loss: 0.4511 - val_accuracy: 0.8421
Epoch 4/10
8/8 [==============================] - 10s 1s/step - loss: 0.4373 - accuracy: 0.7760 - val_loss: 0.3857 - val_accuracy: 0.9474
Epoch 5/10
8/8 [==============================] - 9s 1s/step - loss: 0.3718 - accuracy: 0.8122 - val_loss: 0.2820 - val_accuracy: 0.9737
Epoch 6/10
8/8 [==============================] - 9s 1s/step - loss: 0.3132 - accuracy: 0.8611 - val_loss: 0.1825 - val_accuracy: 0.9474
Epoch 7/10
8/8 [==============================] - 9s 1s/step - loss: 0.2905 - accuracy: 0.8817 - val_loss: 0.1889 - val_accuracy: 0.9737
Epoch 8/10
8/8 [======================

In [ ]:
model.save('covid_19.h5')